# Part 1 - Build a basic Chatbot

## Pre-running - Questions 
1. what's different with calling LLM directly for building a chatbot? 


## Key Concepts on Langgraph
1. langgraph is stateful


## API Reference
[]


In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain-openai

In [2]:
import getpass
import os
from langchain_openai import ChatOpenAI

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## `#key-concept`  
1. Langgraph is stateful graph, so you need to define the `State`
2. Langgraph is stateful graph, so you need to define the template as `StateGraph(State)` 

`Stateful` means it's a typed dictionary
{"messages": [add_messages]}

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

##  chatbot Node
`key-concept`  Node
1. 

In [4]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated

llm = ChatOpenAI()
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [5]:


from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [6]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: Singapore is a small island city-state located in Southeast Asia. It is known for its modern and efficient infrastructure, multicultural society, and diverse food scene. The country is a global financial hub and one of the most developed economies in Asia. Singapore is also famous for its clean streets, strict laws, and beautiful gardens, including the iconic Gardens by the Bay and Marina Bay Sands. Additionally, Singapore is a popular tourist destination, attracting visitors with its shopping malls, luxury hotels, and vibrant nightlife. The country is also home to various cultural and religious landmarks, such as Chinatown, Little India, and the Sultan Mosque.
Goodbye!
